In [1]:
import pandas as pd
import datetime
import pandas_datareader.data as web
import fix_yahoo_finance as yf

In [10]:
end = datetime.date.today()
start = end - datetime.timedelta(days = 365*10)

print(start)

2008-11-24


In [24]:
data = web.get_data_yahoo('AAPL', start,end)

In [27]:
data['Return'] = data['Close'].pct_change()
data = data.dropna()
data

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,High,Low,Open,Close,Volume,Adj Close,Return
Date,,,,,,,
2008-11-26,13.607142,12.835714,12.845715,13.571428,224959000.0,9.088281,0.046255
2008-11-28,13.537143,13.122857,13.528571,13.238571,74443600.0,8.865379,-0.024526
2008-12-01,13.181429,12.702857,13.042857,12.704286,230941900.0,8.507588,-0.040358
2008-12-02,13.235714,12.357142,12.861428,13.210000,287180600.0,8.846247,0.039807
2008-12-03,13.747143,12.685715,12.771428,13.700000,334670000.0,9.174378,0.037093
2008-12-04,13.601429,12.722857,13.490000,13.058572,272842500.0,8.744838,-0.046820
2008-12-05,13.498571,12.694285,12.907143,13.428572,260948800.0,8.992613,0.028334
2008-12-08,14.400000,13.685715,13.897142,14.245714,296285500.0,9.539826,0.060851
2008-12-09,14.800000,13.887143,14.005714,14.294286,300874000.0,9.572351,0.003410


In [29]:
std = data['Return'].describe().loc['std']
std

0.016900457092183272

In [44]:
def get_var(stock_name, start = 0, end = 0):
    import pandas as pd
    import datetime
    import pandas_datareader.data as web
    import fix_yahoo_finance as yf
    end = datetime.date.today()
    start = end - datetime.timedelta(days = 365*10)
    data = web.get_data_yahoo(stock_name, start,end)
    data['Return'] = data['Close'].pct_change()
    data = data.dropna()
    std = data['Return'].describe().loc['std']
    return std

In [45]:
get_var('GS')

0.02059900286455084

In [46]:
import requests
from bs4 import BeautifulSoup

In [100]:
import pandas as pd
url = 'https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yield'
response = requests.get(url)
if response.status_code == 200:
    #print("Success")
    data_page = BeautifulSoup(response.content,'lxml')
    yield_table = data_page.find('table', class_ = 't-chart')
    trs = yield_table.find_all('tr')
    tenor = [trs[0].find_all('th', scope = 'col')[i].get_text() 
           for i in range(1,len(trs[0].find_all('th', scope = 'col')))]
    date = [trs[i].find_all('td', scope = 'row')[0].get_text() for i in range(1,len(trs))]

else:
    print('Failure')
#trs[2].find_all('td')
data = pd.DataFrame(0, columns = tenor, index = date)
data
# first commit

,1 mo,2 mo,3 mo,6 mo,1 yr,2 yr,3 yr,5 yr,7 yr,10 yr,20 yr,30 yr
11/01/18,0,0,0,0,0,0,0,0,0,0,0,0
11/02/18,0,0,0,0,0,0,0,0,0,0,0,0
11/05/18,0,0,0,0,0,0,0,0,0,0,0,0
11/06/18,0,0,0,0,0,0,0,0,0,0,0,0
11/07/18,0,0,0,0,0,0,0,0,0,0,0,0
11/08/18,0,0,0,0,0,0,0,0,0,0,0,0
11/09/18,0,0,0,0,0,0,0,0,0,0,0,0
11/13/18,0,0,0,0,0,0,0,0,0,0,0,0
11/14/18,0,0,0,0,0,0,0,0,0,0,0,0
11/15/18,0,0,0,0,0,0,0,0,0,0,0,0
